이미 만들어 둔, 문서 - 단어 행렬을 이용하여 NMF 를 이용한 토픽모델링을 합니다.

In [1]:
import pickle
with open('../../../data/corpus_10days/models/params_keywords', 'rb') as f:
    params = pickle.load(f)

x = params['x']
vocab2idx = params['word2index']
idx2vocab = params['index2word']

300,91 개의 문서가 9,774 개의 단어로 표현되어 있습니다.

In [2]:
x.shape

(30091, 9774)

i7-5820 기준 7 분의 학습 시간이 필요합니다.

In [3]:
from sklearn.decomposition import NMF

if False:
    
    nmf = NMF(n_components=100)
    y = nmf.fit_transform(x)
    
    with open('./2016-10-20-nmf.pkl', 'wb') as f:
        pickle.dump(nmf, f)
    with open('./2016-10-20-nmf_y.pkl', 'wb') as f:
        pickle.dump(y, f)

else:
    
    with open('./2016-10-20-nmf.pkl', 'rb') as f:
        nmf = pickle.load(f)
    with open('./2016-10-20-nmf_y.pkl', 'rb') as f:
        y = pickle.load(f)

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator NMF from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


9,774 차원으로 표현되는 문서가 100 차원으로 표현됩니다. 

In [4]:
y.shape

(30091, 100)

9,774 개의 단어 역시 100 차원으로 표현됩니다. SVD 처럼 components_ 에 정보가 저장되어 있습니다.

이를 반대로 해석하면 100 개의 components 들이 각각 9,774 개의 term weight vector 로 표현된 것과 같습니다. 30091 개의 문서를 표현할 수 있는 100 개의 components 입니다.

In [5]:
nmf.components_.shape

(100, 9774)

안전하게 components vector 를 복사해둡니다.

In [6]:
components = nmf.components_.copy()

한 component 에 대하여 weight 의 크기가 큰 순서대로 component keyword 를 선택할 수 있습니다. 일종의 labeling 입니다.

In [7]:
def most_important_terms(component, topn=10, decode=True):
    
    # sort by weight in decreasing order
    terms = component.argsort()[::-1]
    
    # select top n terms
    if topn > 0:
        terms = terms[:topn]

    # form of [(idx, weight), ... ]
    weights = component[terms]
    term_and_weights = [(t,w) for t,w in zip(terms, weights)]
    
    # decode
    if decode:
        term_and_weights = [(idx2vocab[t], w) for t,w in term_and_weights]
    
    return term_and_weights

most_important_terms(components[71])

[('불독', 15.720180055497874),
 ('데뷔', 7.436744476677633),
 ('걸그룹', 6.392035159858045),
 ('쇼케이스', 4.529443114555796),
 ('키미', 4.244292664767637),
 ('형은', 4.037779583606048),
 ('무대', 3.4491967317880823),
 ('소라', 3.3814349219963007),
 ('롤링', 3.2202080370965027),
 ('세이', 3.08934217032449)]

한 단어에 대하여 weight 가 큰 components 를 찾을 수도 있습니다.

In [8]:
def most_relavant_components(term, n_components=5, n_terms=20, decode=True):
    term_idx = vocab2idx.get(term, -1)
    if term_idx < 0:
        return []

    # copy components for safety
    components = nmf.components_.copy()
    
    # slice columns of term and sort in decreasing order
    relavant_idx = components[:,term_idx].argsort()[::-1]
    
    # select top n_components
    if n_components > 0:
        relavant_component_idx = relavant_idx[:n_components]

    # select important terms for each component
    relevant_components = []
    for component_idx in relavant_component_idx:
        # idx, score, terms
        relevant_components.append(
            (component_idx,
             components[component_idx, term_idx],
             most_important_terms(components[component_idx], n_terms, decode))
        )
    
    return relevant_components

'아이오아이'라는 단어의 weight 가 큰 components 들의 top weighted words 입니다.

In [9]:
for component_idx, relavant_score, important_terms in most_relavant_components('아이오아이'):
    terms = [term for term, _ in important_terms]
    print('component#{} : {}'.format(
        component_idx,' '.join(terms)))

component#71 : 불독 데뷔 걸그룹 쇼케이스 키미 형은 무대 소라 롤링 세이 오전 마포구 매력 101 멤버들 싱글 프로듀스 강렬 20일 표현
component#11 : 공개 모습 화보 캔디 공유 캐릭터 도깨비 매력 선보 영상 마음 촬영 한편 메이크업 소리 조안 장근석 특히 변신 기대감
component#60 : 기록 1위 트와이스 스트리밍 방탄소년단 차트 누적 발표 뮤직비디오 올해 가온차트 2016년 데뷔 유튜브 최고 미니앨범 조회수 차지 1주년 부문
component#23 : 방송 출연 이날 프로그램 전현무 무대 예능 웃음 오후 시청률 지상파 노래 김지민 아프리카 광고 라고 샤이니 한편 20일 이에
component#79 : 신화 앨범 발매 13집 팬들 정규 컴백 활동 11월 그룹 이번 공개 콘서트 데뷔 예정 멤버들 기대 곡들 19년 아이돌


'트럼프'라는 단어의 weight 가 큰 components 들의 top weighted words 입니다. 

In [10]:
for component_idx, relavant_score, important_terms in most_relavant_components('트럼프'):
    terms = [term for term, _ in important_terms]
    print('component#{} : {}'.format(
        component_idx,' '.join(terms)))

component#1 : 트럼프 클린턴 토론 대선 후보 힐러리 주장 공화당 미국 3차 선거 민주당 도널드 대선후보 결과 라스베이거스 지지 이날 이라고 발언
component#39 : 후보 지금 힐러리 미국 말씀 트럼프 있습니다 이런 때문 제가 우리 사실 클린턴 생각합니다 경제 생각 발언 국가 대통령 도널드
component#57 : 여성 남성 여성들 남성들 혐오 자신 사회 임신 남자 표현 운동 말하는 이상 낙태 차별 사건 비율 생각 여자 군대
component#14 : 중국 필리핀 두테르테 양국 남중국해 베이징 주석 정상회담 분쟁 협력 영유권 시진핑 방문 경제 투자 성장률 갈등 수출 로드 확대
component#42 : 영화 감독 개봉 작품 배우 연기 걷기왕 이야기 관객 관객들 제작 심은경 흥행 만복 주연 출연 배우들 스크린 럭키 영화제


NMF 는 SVD 와 활용하는 방법은 비슷합니다. component 를 학습하는 방식이 다릅니다.